In [1]:
server = '140.114.77.15'
database = 'ChatService_EmotionTest'
username = 'sa'
password = '1Qazwsxedc'
driver= 'FreeTDS'

In [2]:
import pymssql
import codecs
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
import numpy as np
from sklearn import metrics

conn = pymssql.connect(server=server, user=username, password=password, database=database)

In [3]:
cursor = conn.cursor()  
cursor.execute('SELECT @@version;')  
row = cursor.fetchone() 
print(row)

('Microsoft SQL Server vNext (CTP1.3) - 14.0.304.138 (X64) \n\tFeb 13 2017 16:49:12 \n\tCopyright (C) 2016 Microsoft Corporation. All rights reserved.\n\ton Linux (Ubuntu 16.04.2 LTS)',)


In [4]:
cursor = conn.cursor()  
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE'")  
row = cursor.fetchone() 
print(row)
while row:  
    print(str(row[0]) + " " + str(row[1]) + " " + str(row[2]))
    row = cursor.fetchone()

('ChatService_EmotionTest', 'dbo', 'ChatList', 'BASE TABLE')
ChatService_EmotionTest dbo ChatList
ChatService_EmotionTest dbo ChatMessages
ChatService_EmotionTest dbo sysdiagrams


In [11]:
cursor.description

(('ChatId', 1, None, None, None, None, None),
 ('MessageTime', 4, None, None, None, None, None),
 ('SenderId', 1, None, None, None, None, None),
 ('MessageType', 1, None, None, None, None, None),
 ('Content', 1, None, None, None, None, None),
 ('MsgId', 3, None, None, None, None, None))

In [9]:
cursor = conn.cursor()  
cursor.execute("select top 30* from ChatMessages")  
row = cursor.fetchone() 
while row:  
    print(row)
    row = cursor.fetchone()

('2361157b-cc70-4bdb-98f6-d17cd7ba9d33', datetime.datetime(2015, 8, 26, 21, 51, 44, 247000), 'D425218C-3E8D-490B-84E9-737E9C58576C', 'text', '還在嗎？', 1)
('236309e1-2b95-4ab5-906e-929e5c1f30d8', datetime.datetime(2015, 7, 2, 8, 7, 46, 890000), 'EEC5CFA6-5B32-4281-82D1-3FA06C32CA84', 'text', '你好,padfone s更新5.0後,安裝在SD卡的程式都不能使用', 2)
('236309e1-2b95-4ab5-906e-929e5c1f30d8', datetime.datetime(2015, 7, 2, 8, 8, 33, 177000), 'A0110', 'text', '這部份,請您先把這些app移到手機內', 3)
('236309e1-2b95-4ab5-906e-929e5c1f30d8', datetime.datetime(2015, 7, 2, 8, 12, 19, 753000), 'A0110', 'text', '那請您在這邊找到後,移重到手機後,再移除', 4)
('2363f532-23b6-484a-b3cc-a7a7424822ce', datetime.datetime(2015, 9, 7, 8, 25, 34, 353000), 'CE88B5A2-E432-4B88-9DEE-D4CCB9C28484', 'text', '今年五月底購機,之前偶爾會重開機,近日手機不停發生自動重開機,今天光早上八點左右到八點半,已自動重開機三次,非常討厭,請盡速回覆處理。台北王律師', 5)
('22fbb230-9305-4c21-98d8-e7d2877c0d76', datetime.datetime(2016, 9, 29, 9, 48, 59, 263000), 's6262741s84s@gmail.com', 'text', '謝謝', 6)
('22fbc764-a52f-4317-a629-16c7ef51e5be', datetime.

In [6]:
cursor.execute('SELECT top 30* FROM ChatList where Survey is not null')

for row in cursor:
    print('row = %r' % (row,))
cursor.description

row = ('1c8fa7f5-0797-4b09-be5a-bbe48d212e69', 'TW', 'mobile', 'MOBILE', datetime.datetime(2015, 8, 19, 20, 51, 31, 857000), datetime.datetime(2015, 8, 19, 21, 15, 31, 227000), 0, 781, '1', datetime.datetime(2015, 8, 19, 21, 4, 37), None)
row = ('19c07cd6-e059-49ef-94fd-64a46ec10d23', 'TW', 'mobile', 'MOBILE', datetime.datetime(2015, 7, 25, 18, 41, 44, 267000), datetime.datetime(2015, 7, 25, 18, 54, 26, 227000), 0, 759, '2', datetime.datetime(2015, 7, 25, 18, 47, 38, 170000), None)
row = ('19c1466b-10ff-489a-8f60-34c4a1c50a52', 'TW', 'mobile', 'MOBILE', datetime.datetime(2015, 4, 25, 14, 16, 16, 210000), datetime.datetime(2015, 4, 25, 14, 36, 52, 687000), 0, 1233, '2', datetime.datetime(2015, 4, 25, 14, 36, 46, 943000), None)
row = ('19eadf8f-46ae-4d10-ab6c-c8e0e72395a3', 'TW', 'mobile', 'MOBILE', datetime.datetime(2015, 7, 27, 13, 53, 40, 257000), datetime.datetime(2015, 7, 27, 14, 1, 13, 423000), 0, 376, '1', datetime.datetime(2015, 7, 27, 14, 0, 10, 83000), None)
row = ('17ac9c34-74

(('ChatId', 1, None, None, None, None, None),
 ('TenantId', 1, None, None, None, None, None),
 ('Channel', 1, None, None, None, None, None),
 ('Skill', 1, None, None, None, None, None),
 ('CreateTime', 4, None, None, None, None, None),
 ('DisposeTime', 4, None, None, None, None, None),
 ('QueueMS', 3, None, None, None, None, None),
 ('Duration', 3, None, None, None, None, None),
 ('Survey', 1, None, None, None, None, None),
 ('SurveyTime', 4, None, None, None, None, None),
 ('Suggestion', 1, None, None, None, None, None))

In [12]:
cursor.execute('SELECT count(1) FROM ChatList')
cursor.fetchone() 

cursor.execute('SELECT count(1) FROM ChatList WHERE Survey = 1')
cursor.fetchone() 

# import codecs
# cursor.execute('SELECT Content FROM ChatList \
#                JOIN ChatMessages ON ChatList.ChatId = ChatMessages.ChatId \
#                WHERE ChatList.Survey IS NOT NULL AND \
#                LEN(ChatMessages.SenderId) > 5 AND\
#                ChatMessages.MessageType = \'text\'\
#                ORDER BY ChatMessages.ChatId\
#                ')
# f = codecs.open("./test_data/msg.txt","w", "utf-8-sig")
# for msg in cursor.fetchall():
#     f.write(msg[0]+'\n')

(44992,)

In [3]:
cursor = conn.cursor()  
cursor.execute("SELECT ChatList.ChatId, ChatMessages.SenderId , ChatMessages.Content from ChatList \
                JOIN ChatMessages ON ChatList.ChatId = ChatMessages.ChatId \
                WHERE ChatList.Survey = 1 AND ChatMessages.MessageType = \'text\' \
                ORDER BY ChatMessages.ChatId")  
row = cursor.fetchone() 
pos_data_sets = {}
while row:
    if row[0] in pos_data_sets:
        pos_data_sets[row[0]].append(row)
    else:
        pos_data_sets[row[0]] = [row]
    row = cursor.fetchone()

for key in list(pos_data_sets.keys()):
    user_content_array = []
    for data in pos_data_sets[key]:
        if data[1] == None or len(data[1]) > 5:
            user_content_array.append(data[2])
    pos_data_sets[key] = user_content_array

for key in list(pos_data_sets.keys()):
    if len(pos_data_sets[key]) == 0:
        del pos_data_sets[key]


In [91]:
for index, row in enumerate(pos_data_sets):
    if index>20:
        break
    print(pos_data_sets[row])

['不知怎麼設定', 'ZF2', '滿意,謝謝你', '才能收到', '就是MMS沒有圖片', '文字收到', '還是要,放在卡1', '原來如此', '放到sim1', '之前用S3,可以收到mms', '應該不是中華電信的問題', '才能收到']
['那一個小時預約快修 的部分是?', '這樣要怎麼處理呢', '所以就有機會 當日維修好囉', '及最下面三個 案件  也沒反應', 'ZF2', '一個小時快修的部分', ' 恩恩  對', '不是', '如果送修的話 有可能當天就好嗎', '請問螢幕觸控按了沒反應', '沒使用單手模式下   任何情況那三個按鍵都沒反應', '還有螢幕中間部分區塊按了也沒反應', '恩   好   我了解了', '謝謝你', '550ML', '現在我只能轉換成單手操作變成小螢幕的虛擬三個觸控按鈕才能使用那三個案件的功能', '是因為沒辦法按 我才轉換成單手模式的']
['維修地點是板橋皇家沒錯', '瞭解, 感謝您~~~', '您好,請問官網上的維修進度查詢 如果顯示可以去取件了 是否就表示維修完成了呢？', '因為之前都會有簡訊通知 但我一直沒收到 剛剛線上查詢了 發現昨天就可以取件了', '江選帆', 'TWA3610915']
['ZENFONE 2', '你好 可以線上查詢保固日期嗎', 'F5AZFG21A099', '好的 謝謝']
['已傳輸到sd卡了', '檔案要放到那裡?', '我先試試,有問題再來請問您們', 'tks', '如已有解壓會有什麼狀況嗎？', '請問要如何手動更新下載軔體?', '我的手機是 ZenFone6', 'sd卡可以?', '這裡對嗎？', '已解壓縮檔要移除嗎？', 'ok ']
['你好', '我要把目前使用的手機號碼轉到另外一支,出門不用帶二支', '簡單明瞭', '我要將手機轉接,收到錯誤訊息,不能轉接出去', '簡單說我要將我的手機號碼轉至另一支', '我有開網路', 'zf3', '把我的電話轉到另外一支', '二支手機', '但不行', '謝了,了解', '不能轉接到另一支手機', '來電轉接無法轉接', '886', '網路傳回非預期回應', '通話設定==>來電轉接==>一律轉接==>輸入號碼後按開啟,出現網路傳回非預期回應']
['謝謝您', '請問一下', 'z

In [97]:
8135*2

16270

In [98]:
corpus = []
satisfied_degree = []
for index, key in enumerate(pos_data_sets):
#     if index>16270:
#         break
    corpus.append(' '.join(jieba.cut(' '.join(pos_data_sets[key]), cut_all=False)))
    satisfied_degree.append(1)

for key in neg_data_sets:
    corpus.append(' '.join(jieba.cut(' '.join(neg_data_sets[key]), cut_all=False)))
    satisfied_degree.append(-1)

In [100]:
len(corpus)

24406

## 1.先將正面與負面的 使用者字詞 利用TFIDF 算出Bag of word
## 2.再將正負面丟去Train Classier 
## 3.在跟助教給的情緒LIB 比對分數看看



In [101]:
import plotly.graph_objs as go
from plotly.graph_objs import *
import plotly.plotly as py
import plotly.figure_factory as ff
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

np_corpus = np.array(corpus)
np_satisfied_degree = np.array(satisfied_degree)

X_train, X_test, y_train, y_test = train_test_split(
    np_corpus, np_satisfied_degree, test_size=0.3, random_state=0)

# scikit-learn example

In [164]:
print("Creating the bag of words...\n")

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(X_train)
test_data_features = vectorizer.transform(X_test)

print(train_data_features.shape)
print(test_data_features.shape)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
test_data_features = test_data_features.toarray()

Creating the bag of words...

(17084, 500)
(7322, 500)


In [141]:
analyze = vectorizer.build_analyzer()
analyze("華碩 品質 好差")
len(train_data_features[0])

300

In [142]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(train_data_features)
print(X_train_tfidf.shape)
X_test_tfidf = tfidf_transformer.transform(test_data_features)
print(X_test_tfidf.shape)

(17084, 300)
(7322, 300)


In [143]:
# Naive_bayes 
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train)

predicted = clf.predict(X_test_tfidf)

print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

             precision    recall  f1-score   support

          0       0.81      0.15      0.25      2390
          1       0.70      0.98      0.82      4932

avg / total       0.74      0.71      0.63      7322



In [136]:
# SVM
print("Training the SVM...")
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

clf = LinearSVC().fit(X_train_tfidf, y_train)
predicted = clf.predict(X_test_tfidf)
print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

Training the SVM...
             precision    recall  f1-score   support

          0       0.66      0.65      0.65      2390
          1       0.83      0.84      0.83      4932

avg / total       0.77      0.78      0.77      7322



In [144]:
print("Training the DecisionTree...")
from sklearn.tree import DecisionTreeClassifier

# criterion : impurity function
# max_depth : maximum depth of tree
# random_state : seed of random number generator
tree = DecisionTreeClassifier(criterion='entropy', 
                              max_depth=3, 
                              random_state=0)
tree.fit(X_train_tfidf, y_train)
predicted = tree.predict(X_test_tfidf)
print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

Training the DecisionTree...
             precision    recall  f1-score   support

          0       0.61      0.78      0.68      2390
          1       0.88      0.76      0.81      4932

avg / total       0.79      0.77      0.77      7322



In [145]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( X_train_tfidf,  y_train)

predicted = forest.predict(X_test_tfidf)

print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

Training the random forest...
             precision    recall  f1-score   support

          0       0.67      0.65      0.66      2390
          1       0.83      0.84      0.84      4932

avg / total       0.78      0.78      0.78      7322



# Manually Playground

In [62]:
test_data = [['爛公司 華碩 品質 無法 解決 問題'], ['這公司 好爛 惹我生氣 煩欸 幹 無法 解決 問題'], ['不知怎麼設定', 'ZF2', '滿意,謝謝你', '才能收到', '就是MMS沒有圖片', '文字收到', '還是要,放在卡1', '原來如此', '放到sim1', '之前用S3,可以收到mms', '應該不是中華電信的問題', '才能收到'],
['那一個小時預約快修 的部分是?', '這樣要怎麼處理呢', '所以就有機會 當日維修好囉', '及最下面三個 案件  也沒反應', 'ZF2', '一個小時快修的部分', ' 恩恩  對', '不是', '如果送修的話 有可能當天就好嗎', '請問螢幕觸控按了沒反應', '沒使用單手模式下   任何情況那三個按鍵都沒反應', '還有螢幕中間部分區塊按了也沒反應', '恩   好   我了解了', '謝謝你', '550ML', '現在我只能轉換成單手操作變成小螢幕的虛擬三個觸控按鈕才能使用那三個案件的功能', '是因為沒辦法按 我才轉換成單手模式的'],
['維修地點是板橋皇家沒錯', '瞭解, 感謝您~~~', '您好,請問官網上的維修進度查詢 如果顯示可以去取件了 是否就表示維修完成了呢？', '因為之前都會有簡訊通知 但我一直沒收到 剛剛線上查詢了 發現昨天就可以取件了', '江選帆', 'TWA3610915'],
['ZENFONE 2', '你好 可以線上查詢保固日期嗎', 'F5AZFG21A099', '好的 謝謝'],
['已傳輸到sd卡了', '檔案要放到那裡?', '我先試試,有問題再來請問您們', 'tks', '如已有解壓會有什麼狀況嗎？', '請問要如何手動更新下載軔體?', '我的手機是 ZenFone6', 'sd卡可以?', '這裡對嗎？', '已解壓縮檔要移除嗎？', 'ok '],
['你好', '我要把目前使用的手機號碼轉到另外一支,出門不用帶二支', '簡單明瞭', '我要將手機轉接,收到錯誤訊息,不能轉接出去', '簡單說我要將我的手機號碼轉至另一支', '我有開網路', 'zf3', '把我的電話轉到另外一支', '二支手機', '但不行', '謝了,了解', '不能轉接到另一支手機', '來電轉接無法轉接', '886', '網路傳回非預期回應', '通話設定==>來電轉接==>一律轉接==>輸入號碼後按開啟,出現網路傳回非預期回應'],
['謝謝您', '請問一下', 'zenfone2 4G/32G版的可以去大陸使用通話功能嗎？', '因官方的論壇是有說不能使用4G上網,但我不清楚是否可以使用2G或3G功能？', '謝謝您', '好', '好的'],
['我的螢幕變這樣', '要怎麼送修', '好的', '那要怎麼看sn序號', '保固有免費維修嗎', '恩', '黑白的', '那要怎麼送過去', '蘇敬舜、0937365683、台南市善化區中興路117號', '恩', 'Zenfone5', '那維修一次螢幕要多少', '請問一下官網買的保故是多久', '手機電原鍵拖落、螢幕無法顯示無法處控', '好', '我的sn:E9AZCY162071', '好的', '可以指定地方嗎', '好', '官網', '對', '維修大約要多久'],
['哪我請問一下喔  因為我昨天掃毒這個程式都會停在80% 83% 95% 這個是什麼問題啊？', '嗯嗯 商店下載ㄉ', '沒有了 謝謝 有需要 我會在問ㄉ', '謝謝妳 辛苦了 再見:)^^', '我按滿意 :)  ^^  ', '操作？', '那個我想請問一下DIY大師這個程式會不會跟手機系統程式不合然後造成程式當機？', '為什麼這個程每次掃毒都停在80% 不然就是83% 95%呢？', '妳好', '不好意思我想請問一下', '好 等我一下喔', '可是手機常常重新開機會不會影響手機的程式以及壽命？', 'Zenfone2', '是說掃毒嗎？', '可以了:)', '太久沒重開機？  意思是說 手機過一段時間要重課開的意思？', '好 我知道了 謝謝喔 ^^', '是因為程式太多嗎？', '剛剛上一個客服叫我把全部程式快取清楚我已經用了 現在要怎麼辦？', '恩好 我知道了  謝謝妳:)  ^^', ':) ^^'],
['我要問的是,平常的變壓器多數為1a /2a', '好的', '你好,', '我上面的問題,請問您看得到嗎', '我知道Asus買手機附送的變壓器,是輸出1.5A', '謝謝您的回答', '我知道。當然每個廠商都會建議用原廠的', '我想請問,我可不可以使用變壓器2A 的部分為手機充電呢？', '我的手機是zenfone6'],
['謝謝', '您好 請問高雄哪裡有展示ZenFone 3呢', '沒有了', '嗯嗯 好 '],
['你好我啟動安全後', '哈囉？', '好的', '還是打不開內鏡', '可是我過保固兩個月了', '這樣如果維修鏡頭費用大概會多少', '謝謝', '所以沒有辦法知道換鏡的大概報價嗎？'],
['途中突然不能動,螢幕背景變成黑色', '我有定期檢查是否有更新', '不好意思剛剛打錯網址了', '重開機時會一直震動', '之後才能正常開機', '這樣是該如何處理', '有', '不好意思我不太知道在哪裡', '請問在使用手機時', '之後過了一會兒全暗了', '都不能動', 'zenfone 5', '要去哪裡更新', '對應版本', '.78?'],
['啊羅哈,傑夫', '喔,好,', '我想問,除了網路商店哪裡可以買到實體原廠充電線', 'z5', '喔,好吧,那之後會有嗎', '好的', '只有1小時快修,需要拿到皇家現場嗎', '請問有原廠快速充電嗎', '只有一組 線+充電器嗎,還是有分開單售', '有價錢嗎'],
['我想知道ASUS ZenFone 3 (ZE552KL)目前購買有何優惠', '我想知道ASUS ZenFone 3 (ZE552KL)目前購買有何優惠'],
['另外問單曲最長能放多久?', '單曲無限制?', '1600ms是多九?', '音樂播放時間'],
['好的謝謝您😆', '請問要怎麼關閉zenlife'],
['只要三四個影片就當,為何用蘋果4不會貴公司手機才發生,所以想了解,應該跟line無關吧！', '那些資訊', 'line傳拍影片', '李國慶,0928284789 kuoching86841478@yahoo.com.tw何謂機台,板本', '感謝', '雲端的檔案下載是否會比較慢,我有兩個雲端硬碟華碩,及Google+。很久就反應,覺得是否運算速度不夠', '手機才買半年,客服及時通', '我希望支持國貨,才希望能找出毛病出在那,', '不在旁邊要找 下次如何聯絡', 'ZenFone2', '以前用蘋果不會,應該沒有。可以幫我查', '怎麼找', '為何上傳多張影片,會要重新傳,已經反應多次,還是無解。', '上傳太久就要重來  551', '是否這個', '感謝,掰掰'],
['這問題哪時會解決', '升級為6.0.1後,如何把應用呈式移到SD卡內', '那要如何處理', '還有其他辦法可移嗎'],
['請問台中市有直營服務站嗎？', '嗯', '沒有,我自行過去謝謝'],
['我的zenfone2會有閃屏的狀況', '那該怎麼確定自己的手機還在保固內呢', '面板的畫面會閃爍,有時還有停在特定畫面的狀況', '您好,我有手機方面的問題', '好的,謝謝您~']]
test_data_X = []
for data in test_data:
    test_data_X.append(' '.join(jieba.cut(' '.join(data), cut_all=False)))
test_data_Y = np.ones(21)
test_data_Y = np.append([-1, -1], test_data_Y)
test_data_X = np.array(test_data_X)

In [66]:
y_test

array([ 1,  1, -1, ...,  1,  1, -1])

In [159]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(
                             analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 500)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', RandomForestClassifier(n_estimators = 100)),
])

text_clf = text_clf.fit( X_train,  y_train)


predicted = text_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

             precision    recall  f1-score   support

          0       0.68      0.65      0.66      2390
          1       0.83      0.85      0.84      4932

avg / total       0.78      0.79      0.78      7322



In [162]:
predicted = text_clf.predict(test_data_X)
print(metrics.classification_report(test_data_Y, predicted, target_names=['0', '1']))

             precision    recall  f1-score   support

          0       0.50      1.00      0.67         2
          1       1.00      0.90      0.95        21

avg / total       0.96      0.91      0.93        23



In [163]:
text_clf.predict_proba(test_data_X)

array([[ 0.87      ,  0.13      ],
       [ 0.85      ,  0.15      ],
       [ 0.2       ,  0.8       ],
       [ 0.2       ,  0.8       ],
       [ 0.12      ,  0.88      ],
       [ 0.01      ,  0.99      ],
       [ 0.11      ,  0.89      ],
       [ 0.59      ,  0.41      ],
       [ 0.14      ,  0.86      ],
       [ 0.45      ,  0.55      ],
       [ 0.1       ,  0.9       ],
       [ 0.12      ,  0.88      ],
       [ 0.188     ,  0.812     ],
       [ 0.02      ,  0.98      ],
       [ 0.38      ,  0.62      ],
       [ 0.49708661,  0.50291339],
       [ 0.4075    ,  0.5925    ],
       [ 0.51713974,  0.48286026],
       [ 0.105     ,  0.895     ],
       [ 0.15      ,  0.85      ],
       [ 0.21416667,  0.78583333],
       [ 0.08      ,  0.92      ],
       [ 0.12      ,  0.88      ]])

# Pretraining with TFIDF 

In [102]:
def generate_bar_chart(input_list, chart_title):
    x_axis = []
    y_axis = []
    for item in input_list:
        x_axis.append(item[0])
        y_axis.append(item[1])
    data = [go.Bar(
                x = x_axis,
                y = y_axis
        )]

    layout = go.Layout(
        title = chart_title,
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='basic-bar')

In [103]:
def display_scores(vectorizer, tfidf_result):
    # http://stackoverflow.com/questions/16078015/
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    index_scores = []
    for index, item in enumerate(sorted_scores):
        index_scores.append((index, item[1]))
        
    generate_bar_chart(index_scores, 'TFIDF Score')
    return sorted_scores

In [130]:
# Utilizing TFIDF to delete useless words.
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(corpus)
tfidf_features = tfidf_vectorizer.get_feature_names()

In [131]:
sorted_scores = display_scores(tfidf_vectorizer, tfidf)


/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/site-packages/plotly/plotly/plotly.py:218: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



High five! You successfuly sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~phejimlin/0 or inside your plot.ly account where it is named 'basic-bar'


/home/phejimlin/anaconda3/envs/tensorflow_3.5/lib/python3.5/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow



In [156]:
#Bag of words to select top n = 400
bag_of_words = [word[0] for word in sorted_scores[:401]]

In [157]:
print("Creating the bag of words...\n")

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(vocabulary=bag_of_words) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(X_train)
test_data_features = vectorizer.transform(X_test)

print(train_data_features.shape)
print(test_data_features.shape)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
test_data_features = test_data_features.toarray()

Creating the bag of words...

(17084, 401)
(7322, 401)


In [158]:
print("Training the random forest...")
from sklearn.ensemble import RandomForestClassifier

# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit(train_data_features,  y_train)

predicted = forest.predict(test_data_features)

print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

Training the random forest...
             precision    recall  f1-score   support

          0       0.67      0.63      0.65      2390
          1       0.83      0.85      0.84      4932

avg / total       0.78      0.78      0.78      7322



In [174]:
text_clf = Pipeline([('vect', CountVectorizer(vocabulary=bag_of_words)),
                      ('clf', RandomForestClassifier(n_estimators = 100)),
])

text_clf = text_clf.fit(X_train,  y_train)


predicted = text_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))
print(text_clf.predict_proba(test_data_X))

             precision    recall  f1-score   support

          0       0.67      0.62      0.64      2390
          1       0.82      0.85      0.84      4932

avg / total       0.77      0.78      0.77      7322

[[ 0.9         0.1       ]
 [ 0.94        0.06      ]
 [ 0.1         0.9       ]
 [ 0.31        0.69      ]
 [ 0.09        0.91      ]
 [ 0.005       0.995     ]
 [ 0.24        0.76      ]
 [ 0.44        0.56      ]
 [ 0.19        0.81      ]
 [ 0.39        0.61      ]
 [ 0.16        0.84      ]
 [ 0.08        0.92      ]
 [ 0.05        0.95      ]
 [ 0.05        0.95      ]
 [ 0.56        0.44      ]
 [ 0.46485714  0.53514286]
 [ 0.33        0.67      ]
 [ 0.72437121  0.27562879]
 [ 0.24616667  0.75383333]
 [ 0.22        0.78      ]
 [ 0.28333333  0.71666667]
 [ 0.          1.        ]
 [ 0.13        0.87      ]]


In [175]:
import pickle
# now you can save it to a file
with open('TFIDF_RandomForestClassifier.pkl', 'wb') as f:
    pickle.dump(text_clf, f)

In [172]:
# Naive_bayes 
Multinomial_clf = Pipeline([('vect', CountVectorizer(vocabulary=bag_of_words)),
                      ('clf', MultinomialNB()),
])
Multinomial_clf = Multinomial_clf.fit(X_train, y_train)

predicted = Multinomial_clf.predict(X_test)

print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

             precision    recall  f1-score   support

          0       0.62      0.46      0.53      2390
          1       0.77      0.86      0.81      4932

avg / total       0.72      0.73      0.72      7322



In [173]:
with open('Naive_bayes_Classifier.pkl', 'wb') as f:
    pickle.dump(Multinomial_clf, f)

In [168]:
# SVM
print("Training the SVM...")
LinearSVC_clf = Pipeline([('vect', CountVectorizer(vocabulary=bag_of_words)),
                      ('clf', LinearSVC()),
])

LinearSVC_clf = LinearSVC_clf.fit(X_train, y_train)
predicted = clf.predict(X_test)
print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

Training the SVM...
             precision    recall  f1-score   support

          0       0.62      0.46      0.53      2390
          1       0.77      0.86      0.81      4932

avg / total       0.72      0.73      0.72      7322



In [170]:
with open('LinearSVC_Classifier.pkl', 'wb') as f:
    pickle.dump(LinearSVC_clf, f)

In [169]:
print("Training the DecisionTree...")
DecisionTree_clf = Pipeline([('vect', CountVectorizer(vocabulary=bag_of_words)),
                      ('clf', DecisionTreeClassifier(criterion='entropy', 
                              max_depth=3, 
                              random_state=0)),
])

# criterion : impurity function
# max_depth : maximum depth of tree
# random_state : seed of random number generator
DecisionTree_clf=DecisionTree_clf.fit(X_train, y_train)
predicted = DecisionTree_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

Training the DecisionTree...
             precision    recall  f1-score   support

          0       0.61      0.78      0.69      2390
          1       0.88      0.76      0.82      4932

avg / total       0.79      0.77      0.77      7322



In [171]:
with open('DecisionTree_Classifier.pkl', 'wb') as f:
    pickle.dump(DecisionTree_clf, f)

# load modle with pickle

In [176]:
# and later you can load it
with open('TFIDF_RandomForestClassifier.pkl', 'rb') as f:
    clf = pickle.load(f)

In [177]:
print(clf.predict_proba(test_data_X))
predicted = clf.predict(X_test)
print(metrics.classification_report(y_test, predicted, target_names=['0', '1']))

[[ 0.9         0.1       ]
 [ 0.94        0.06      ]
 [ 0.1         0.9       ]
 [ 0.31        0.69      ]
 [ 0.09        0.91      ]
 [ 0.005       0.995     ]
 [ 0.24        0.76      ]
 [ 0.44        0.56      ]
 [ 0.19        0.81      ]
 [ 0.39        0.61      ]
 [ 0.16        0.84      ]
 [ 0.08        0.92      ]
 [ 0.05        0.95      ]
 [ 0.05        0.95      ]
 [ 0.56        0.44      ]
 [ 0.46485714  0.53514286]
 [ 0.33        0.67      ]
 [ 0.72437121  0.27562879]
 [ 0.24616667  0.75383333]
 [ 0.22        0.78      ]
 [ 0.28333333  0.71666667]
 [ 0.          1.        ]
 [ 0.13        0.87      ]]
             precision    recall  f1-score   support

          0       0.67      0.62      0.64      2390
          1       0.82      0.85      0.84      4932

avg / total       0.77      0.78      0.77      7322

